# RAG Quickstart for Azure AI Search

This quickstart provides a query for RAG scenarios. It demonstrates an approach for a chat experience using grounding data from a search index on Azure AI Search.

We took a few shortcuts to keep the exercise basic and focused on query definitions:

- We use the **hotels-sample-index**, which can be created in minutes and runs on any search service tier. This index is created by a wizard using built-in sample data.

- We omit vectors so that we can skip chunking and embedding. The index contains plain text.

A non-vector index isn't ideal for RAG patterns, but it makes for a simpler example.

Once you understand the fundamentals of integrating queries from Azure AI Search to a chat completion model on Azure OpenAI, you can build on that experience by adding vector fields and vector and hybrid queries. We recommend the [phi-chat Python code example](https://github.com/Azure/azure-search-vector-samples/blob/main/demo-python/code/phi-chat/phi-chat.ipynb) for that step.

This example is fully documented in [Quickstart: Generative search (RAG) with grounding data from Azure AI Search](https://learn.microsoft.com/azure/search/search-get-started-rag). If you need more guidance than the readme provides, please refer to the article.


## Prerequisites

- [Azure OpenAI](https://learn.microsoft.com/azure/ai-services/openai/how-to/create-resource)

  - [Choose a region](https://learn.microsoft.com/azure/ai-services/openai/concepts/models?tabs=global-standard%2Cstandard-chat-completions#global-standard-model-availability) that supports the chat completion model you want to use (gpt-4o, gpt-4o-mini, or equivalent model). 
  - [Deploy the chat completion model](https://learn.microsoft.com/azure/ai-studio/how-to/deploy-models-openai) in Azure AI Foundry or [use another approach](https://learn.microsoft.com/azure/ai-services/openai/how-to/working-with-models?tabs=powershell).


- [Azure AI Search](https://learn.microsoft.com/azure/search/search-create-service-portal)

  - Basic tier or higher is recommended.
  - Enable semantic ranking.
  - Enable role-based access control.
  - Enable a system identity for Azure AI Search.
  
Make sure you know the name of the deployed model, and have the endpoints for both Azure resources at hand. You will provide this information in the steps that follow.

## Configure access

This quickstart assumes authentication and authorization using Microsoft Entra ID and role assignments. It also assumes that you run this code from your local device.

1. To create, load, and query the sample hotels index on Azure AI Search, you must personally have role assignments for **Search Index Data Contributor** and **Search Service Contributor**. If the hotels index already exists, you only need **Search Index Data Reader**.

1. To send the query and search results to Azure OpenAI, you must have **Cognitive Services OpenAI User** permissions on Azure OpenAI.

## Create the sample index

This quickstart assumes the hotels-sample-index, which you can create in minutes using [this quickstart](https://learn.microsoft.com/azure/search/search-get-started-portal).

Once the index exists, modify it in the Azure portal to use this semantic configuration:

Now that you have your Azure resources, an index, and model in place, you can run the script to chat with the index.

## Create a virtual environment

Create a virtual environment so that you can install the dependencies in isolation.

1. In Visual Studio Code, open the folder containing Quickstart-RAG.ipynb.

1. Press Ctrl-shift-P to open the command palette, search for "Python: Create Environment", and then select `Venv` to create a virtual environment in the current workspace.

1. Select Quickstart-RAG\requirements.txt for the dependencies.

It takes several minutes to create the environment. When the environment is ready, continue to the next step.

## Log in to Azure

You're using Microsoft Entra ID and role assignments for the connection. Make sure you're logged in to the same tenant and subscription as Azure AI Search and Azure OpenAI. You can use the Azure CLI on the command line to show current properties, change properties, and to log in. For more information, see [Connect without keys](https://learn.microsoft.com/azure/search/search-get-started-rbac). 

Run each of the following commands in sequence.

```
az account show

az account set --subscription <PUT YOUR SUBSCRIPTION ID HERE>

az login --tenant <PUT YOUR TENANT ID HERE>
```

You should now be logged in to Azure from your local device.

## Run the code

In [1]:
# Package install for quickstart
!pip install azure-search-documents --quiet
!pip install azure-identity --quiet
!pip install openai --quiet 


[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip

[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: /opt/homebrew/opt/python@3.11/bin/python3.11 -m pip install --upgrade pip


In [ ]:
# Set endpoints and deployment model (provide the name of the deployment)
AZURE_SEARCH_ENDPOINT = ""
AZURE_OPENAI_ENDPOINT = ""
AZURE_DEPLOYMENT_MODEL = "dev-gpt-4.1-mini"  # e.g., "gpt-35-turbo", "gpt-4o", etc.
AZURE_SEARCH_API_KEY = ""
AZURE_OPENAI_API_KEY = ""

## Basic RAG Query

The following code demonstrates how to set up a query on string fields and string collections in a search index, and how to send the results to a chat completion model. The response returned to the user is from the chat completion model.

In [ ]:
# Set up the query for generating responses
from azure.search.documents import SearchClient
from azure.core.credentials import AzureKeyCredential
from azure.core.exceptions import HttpResponseError, ClientAuthenticationError
from openai import AzureOpenAI
import sys

try:
    search_credential = AzureKeyCredential(AZURE_SEARCH_API_KEY)

    # Initialize the OpenAI client
    openai_client = AzureOpenAI(api_version="2024-06-01",
                                azure_endpoint=AZURE_OPENAI_ENDPOINT,
                                api_key=AZURE_OPENAI_API_KEY,
    )

    # Initialize the Search client
    search_client = SearchClient(endpoint=AZURE_SEARCH_ENDPOINT,
                                 index_name="hotels-sample-index",
                                 credential=search_credential)
    
except ClientAuthenticationError as auth_error:
    print("인증 오류가 발생합니다. API 키와 엔드포인트를 확인하세요.")
    print(auth_error.message)
    sys.exit(1)
except HttpResponseError as http_error:
    print("HTTP 응답 오류가 발생합니다.")
    print(http_error.message)
    sys.exit(1)
except Exception as e:
    print("알 수 없는 오류가 발생합니다.")
    print(e.message)
    sys.exit(1)


# This prompt provides instructions to the model
GROUNDED_PROMPT="""
You are a friendly assistant that recommends hotels based on activities and amenities.
Answer the query using only the sources provided below in a friendly and concise bulleted manner.
Answer ONLY with the facts listed in the list of sources below.
If there isn't enough information below, say you don't know.
Do not generate answers that don't use the sources below.
Query: {query}
Sources:\n{sources}
"""

# Query is the question being asked. It's sent to the search engine and the LLM.
query="Can you recommend a few hotels with complimentary breakfast?"

try:
    # Search results are created by the search client.
    # Search results are composed of the top 5 results and the fields selected from the search index.
    search_results = search_client.search(
        search_text=query,
        top=5,
        select="Description,HotelName,Tags"
    )

    search_results_list = list(search_results)
    print(f"Found {len(search_results_list)} documents matching the query.")
except Exception as e:
    print("Error during search:")
    print(e.message)
    sys.exit(1)




Found 5 documents matching the query.
Double Sanctuary Resort:5 star Luxury Hotel - Biggest Rooms in the city. #1 Hotel in the area listed by Traveler magazine. Free WiFi, Flexible check in/out, Fitness Center & espresso in room.:['view', 'pool', 'restaurant', 'bar', 'continental breakfast']
Lion's Den Inn:Full breakfast buffet for 2 for only $1. Excited to show off our room upgrades, faster high speed WiFi, updated corridors & meeting space. Come relax and enjoy your stay.:['laundry service', 'free wifi', 'restaurant']
Starlight Suites:Complimentary Airport Shuttle & WiFi. Book Now and save - Spacious All Suite Hotel, Indoor Outdoor Pool, Fitness Center, Florida Green certified, Complimentary Coffee, HDTV:['pool', 'coffee in lobby', 'free wifi']
Royal Cottage Resort:Your home away from home. Brand new fully equipped premium rooms, fast WiFi, full kitchen, washer & dryer, fitness center. Inner courtyard includes water features and outdoor seating. All units include fireplaces and small

If you get an authorization error instead of results:

+ If you just enabled role assignments, wait a few minutes and try again. It can take several minutes for role assignments to become operational.

+ Make sure you [enabled RBAC](https://learn.microsoft.com/azure/search/search-security-enable-roles?tabs=config-svc-portal%2Cdisable-keys-portal) on Azure AI Search. An HttpStatusMessage of **Forbidden** is an indicator that RBAC isn't enabled.

+ Recheck role assignments if you get a 401 error. You should also review the steps in [Connect without keys](https://learn.microsoft.com/azure/search/search-get-started-rbac).

+ Check firewall settings. This quickstart assumes public network access. If you have a firewall, you need to add rules to allow inbound requests from your device and for service-to-service connections. For more information, see [Configure network access](https://learn.microsoft.com/azure/search/service-configure-firewall).

+ For more debugging guidance, see the [troubleshooting section](https://learn.microsoft.com/azure/search/search-get-started-rag#troubleshooting-errors) in the Quickstart documentation.

## Complex RAG Query

Use complex types and collections in a RAG pattern by converting the result to JSON before sending it to the chat completion models. This query is similar to the basic query, but it includes fields from the "Address" complex type and the "Rooms" complex collection. This query shows you how to pull in data from parts of the index that aren't just simple text fields.

In [ ]:
import json

# Query is the question being asked. It's sent to the search engine and the LLM.
query="Can you recommend a few hotels that offer complimentary breakfast? Tell me their description, address, tags, and the rate for one room they have which sleep 4 people."

# Set up the search results and the chat thread.
# Retrieve the selected fields from the search index related to the question.
selected_fields = ["HotelName","Description","Address","Rooms","Tags"]
search_results = search_client.search(
    search_text=query,
    top=5,
    select=selected_fields,
    query_type="simple"
)
sources_filtered = [{field: result[field] for field in selected_fields} for result in search_results]
sources_formatted = "\n".join([json.dumps(source) for source in sources_filtered])

print(sources_formatted)

response = openai_client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": GROUNDED_PROMPT.format(query=query, sources=sources_formatted)
        }
    ],
    model=AZURE_DEPLOYMENT_MODEL
)
 
print(response.choices[0].message.content)

{"HotelName": "Old Century Hotel", "Description": "The hotel is situated in a nineteenth century plaza, which has been expanded and renovated to the highest architectural standards to create a modern, functional and first-class hotel in which art and unique historical elements coexist with the most modern comforts. The hotel also regularly hosts events like wine tastings, beer dinners, and live music.", "Address": {"StreetAddress": "140 University Town Center Dr", "City": "Sarasota", "StateProvince": "FL", "PostalCode": "34243", "Country": "USA"}, "Rooms": [{"Description": "Suite, 2 Double Beds (Mountain View)", "Description_fr": "Suite, 2 lits doubles (vue sur la montagne)", "Type": "Suite", "BaseRate": 250.99, "BedOptions": "2 Double Beds", "SleepsCount": 2, "SmokingAllowed": false, "Tags": ["Room Tags"]}, {"Description": "Standard Room, 1 Queen Bed (City View)", "Description_fr": "Chambre Standard, 1 grand lit (vue ville)", "Type": "Standard Room", "BaseRate": 121.99, "BedOptions": 